In [132]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from pyspark.sql import SQLContext

In [133]:
spark = SparkContext(master='local', appName='Dataframes')
sqlContext = SQLContext(spark)

In [134]:
spark

<SparkContext master=local appName=Dataframes>

In [135]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [136]:
#aca cargo la ruta del archivo
path = "files/"

In [137]:
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line : line.split(','))

In [138]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [139]:
#sc.stop() #stop detiene el contexto

In [140]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [141]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [142]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [143]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: 'ARG' in l)

In [144]:
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [145]:
equiposOlimpicosRDD.count()

1185

In [146]:
equiposOlimpicosRDD.countApprox(20)

1185

In [147]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [174]:
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

In [175]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [176]:
deportistaOlimpicoRDD.count()

135572

In [177]:
deportistaOlimpicoRDD.take(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199']]

In [178]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [179]:
def eliminaEncabezado(indice, interador):
    return iter(list(interador)[1:])

In [180]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [181]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [182]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: 
    (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
    )
)

In [183]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [166]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False )
])

In [167]:
sqlContext.createDataFrame(deportistaOlimpicoRDD, schema).show(3)

Py4JJavaError: An error occurred while calling o1705.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 1 times, most recent failure: Lost task 0.0 in stage 16.0 (TID 18, 192.168.0.16, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-156-db690798eafc>", line 2, in <lambda>
TypeError: int() takes at most 2 arguments (7 given)

	at org.apache.spark.api.python

Py4JJavaError: An error occurred while calling o1652.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 17, 192.168.0.16, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/anthony/spark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-156-db690798eafc>", line 2, in <lambda>
TypeError: int() takes at most 2 arguments (7 given)

	at org.apache.spark.api.python